# **ETL Reviews**

<div style="text-align: justify">
En este notebook de Python, se detallarán los pasos necesarios para llevar a cabo un proceso de ETL (Extract, Transform, Load) en un dataframe que denominaremos "reviews". Los datos que utilizaremos provienen originalmente de un archivo JSON llamado "australian_users_review.json". A lo largo de este documento, se explorarán las etapas de extracción, transformación y carga de datos con el objetivo de preparar el conjunto de datos para su posterior análisis y uso en aplicaciones o modelos de aprendizaje automático.
</div>

## 1. Importación de las librerias

<div style="text-align: justify">
Importar bibliotecas en un ETL es esencial para aprovechar las funcionalidades predefinidas que simplifican tareas como la extracción, transformación y carga de datos. Esto ahorra tiempo y esfuerzo, asegurando un desarrollo eficiente y la creación de procesos ETL más sólidos y confiables.
</div>

In [25]:
import pandas as pd
import funciones_basicas as fb
import pyarrow as pa  
import pyarrow.parquet as pq
from datetime import datetime, timedelta
import ast

## 2. Carga de los datos

En estas lineas de código cargaremos los datos y creamos el dataframe de pandas en el que vamos a realizar el proceso de ETL, todo basado en un arhivo llamado *australian_users_review.json*, como presenta una estructura diferente a nuestro primer dataframe, usaremos otro metodo para abirlo

In [14]:
datos_filas = []

with open('Datasets\\australian_user_reviews.json', 'r', encoding='utf-8') as archivo:  
    for linea in archivo:  
        datos_filas.append(ast.literal_eval(linea))

reviews = pd.DataFrame(datos_filas)
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


## 3. Transformaciones

<div style='text-align: justify'>
En esta sección realizaremos las primeras transformaciones, estas transformaciones se realizan como una buena practica y son transversales a cualquier tipo de datos no importa su contenido, posterior a esas transformacioens inspeccionaremos a fondo la información del dataframe y realizaremos todos los procesos correspondientes, partiendo de ahi. 
</div>

In [15]:
fb.porcentaje_datos_vacios(reviews)

Porcentaje de datos vacíos por columna en el DataFrame:
user_id: 0.00%
user_url: 0.00%
reviews: 0.00%
Total de datos vacíos en el DataFrame: 0.00%


In [16]:
datos_editados = reviews.explode('reviews')  # Divide las listas anidadas dentro de la columna 'review' en filas separadas.
datos_normalizados = pd.json_normalize(datos_editados['reviews'].dropna())  # Normaliza los datos JSON anidados en la columna 'reviews'.
datos_normalizados.reset_index(inplace=True)  # Reinicia los índices del DataFrame normalizado.
datos_editados.reset_index(inplace=True)  # Reinicia los índices del DataFrame editado.

reviews = pd.concat([datos_editados, datos_normalizados], axis=1)  # Concatena los DataFrames editados y normalizados a lo largo de las columnas.
reviews = reviews.drop(columns=['reviews', 'index'])  # Elimina las columnas 'reseñas' e 'index' del DataFrame procesado.

In [17]:
# Normalizamos el texto del dataframe
reviews = reviews.map(fb.normalizar_texto)

In [18]:
# Tratamiento de los datos duplicados y nulos
reviews = reviews.drop_duplicates(keep='first')
reviews = reviews.dropna().reset_index(drop=True)

In [19]:
# Manejo de la columan fecha
reviews['posted'] = reviews['posted'].str.replace('POSTED', '') # Eliminamos la palabra posted de los valores de la columna

# Extraer el año de la columna 'posted'
reviews['posted'] = reviews['posted'].str.extract(r'(\d{4})')

# Convertir el año a formato de fecha %y (año de dos dígitos)
reviews['posted'] = pd.to_datetime(reviews['posted'], errors='coerce').dt.strftime('%Y')

# Rellenar los valores NaN con método de interpolación
reviews['posted'] = reviews['posted'].astype(float)  # Convertir a float para que la interpolación funcione correctamente
reviews['posted'] = reviews['posted'].interpolate(method='linear', limit_direction='both')
reviews['posted'] = reviews['posted'].astype(int)  # Convertir de nuevo a int después de la interpolación

In [20]:
# Aplica la función de análisis de sentimiento a la columna 'reseña' y almacena los resultados en una nueva columna 'analisis_sentimiento'.
reviews["analisis_sentimiento"] = reviews["review"].astype(str).apply(fb.analizar_sentimiento)

In [21]:
# Elimina las columnas irrelevantes del DataFrame procesado.
reviews = reviews.drop(columns=["review", "funny", "last_edited", "helpful", "user_url"])

In [22]:
fb.porcentaje_datos_vacios(reviews)

Porcentaje de datos vacíos por columna en el DataFrame:
user_id: 0.00%
posted: 0.00%
item_id: 0.00%
recommend: 0.00%
analisis_sentimiento: 0.00%
Total de datos vacíos en el DataFrame: 0.00%


In [23]:
reviews.head()

,user_id,posted,item_id,recommend,analisis_sentimiento
0,76561197970982479,2011,1250,True,2
1,76561197970982479,2011,22200,True,1
2,76561197970982479,2011,43110,True,2
3,JS41637,2014,251610,True,2
4,JS41637,2013,227300,True,2


In [24]:
nuevos_nombres = {'user_id':'ID_USUARIO',
                  'posted':'AÑO_LANZAMIENTO',
                  'item_id':'ID_GAME',
                  'recommend':'RECOMIENDA',
                  'analisis_sentimiento':'SENTIMIENTO'}

reviews = reviews.rename(columns=nuevos_nombres)
reviews.head(3)

,ID_USUARIO,AÑO_LANZAMIENTO,ID_GAME,RECOMIENDA,SENTIMIENTO
0,76561197970982479,2011,1250,True,2
1,76561197970982479,2011,22200,True,1
2,76561197970982479,2011,43110,True,2


## 4. Guardamos el archivo

<div style="text-align: justify">
Despues de realizadas todas las transformaciones necesarias para manejar el dataframe en futuros analisis, procedemos a guardar el documento en un formato tipo parquet para que su lectura y escritura sea más sencilla
</div>

In [26]:
fb.guardar_datos(reviews, "PARQUET\\REVIEWS.parquet")